# Coordinate Exchange for I-Optimal Designs

## Packages

In [ ]:
# Set the number of processes to use for parallel computing
using Distributed
addprocs(14)

In [3]:
using LinearAlgebra
using HDF5
using Dates

@everywhere include("../model_builder/model_builder.jl")
@everywhere using .ModelBuilder

@everywhere include("../model_builder/design_initializer.jl")
@everywhere using .DesignInitializer

@everywhere include("./design_optimizer.jl")
@everywhere using .DesignOptimizer

@everywhere include("../cexch/cexch.jl")
@everywhere using .CEXCH

@everywhere include("./optimality_criterion.jl")
@everywhere using .OptimalityCriterion

@everywhere using IterTools: product

## Run

In [3]:
@everywhere NUM_INITIALIZATIONS = 12
@everywhere DESIGNS_PER_INITIALIZATION = 5000
@everywhere MAX_ITERS = 50
@everywhere NUM_SAMPLES = 100
@everywhere Ns = 6:9
@everywhere Ks = [3]
@everywhere models = ["scheffe_2", "special_cubic"]
@everywhere model_builders = [ModelBuilder.scheffe(2), ModelBuilder.special_cubic]
@everywhere obj_crits = [DesignOptimizer.i_criterion(model_builders[i], model=models[i]) for i in 1:length(models)]

Base.Iterators.ProductIterator{Tuple{UnitRange{Int64}, Vector{Int64}, Base.Iterators.Zip{Tuple{Vector{String}, Vector{Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}}, Vector{Main.DesignOptimizer.var"#10#11"{String, Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}}}}}}}((6:10, [3], zip(["scheffe_2", "special_cubic"], Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}[Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}(1, 1, false, Any[], false, true), Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}(1, 2, false, Any[], false, true)], Main.DesignOptimizer.var"#10#11"{String, Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}}[Main.DesignOptimizer.var"#10#11"{String, Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}}("scheffe_2", Main.Model

In [18]:
# Function to be executed in parallel for different design initializations
@everywhere function optimize_designs(designs, obj_crit, max_iters, num_samples)
    CEXCH.cexch(designs, obj_crit, max_iters=max_iters, num_samples=num_samples)
end

# Optimize NxK designs w.r.t. a specific model and objective criterion
function cexch_exp(N, K, model, model_builder, obj_crit)
    # Get initializer function
    init = DesignInitializer.initializer(N, K, model_builder, type="mixture")

    # Initialize designs, will be shape (NUM_INITIALIZATIONS, DESIGNS_PER_INITIALIZATION, N, K)
    design_inits = [init(DESIGNS_PER_INITIALIZATION) for _ in 1:NUM_INITIALIZATIONS]

    # Optimize designs in parallel
    results = pmap(x -> optimize_designs(x, obj_crit, MAX_ITERS, NUM_SAMPLES), design_inits)
    opt_designs, meta = vcat([r[1] for r in results]...), vcat([r[2] for r in results]...)

    return opt_designs, meta
end

cexch_exp (generic function with 2 methods)

In [19]:
# Run jobs
jobs = product(Ns, Ks, zip(models, model_builders, obj_crits))
println("Starting $(length(jobs)) jobs at $(Dates.now())")
for job in jobs
    N, K, (model, model_builder, obj_crit) = job
    opt_designs, meta = cexch_exp(N, K, model, model_builder, obj_crit)
    
    # Persist to file
    h5write("test.h5", "designs_$(model)_$(N)_$(K)", opt_designs)
    h5write("test.h5", "meta_$(model)_$(N)_$(K)", meta)
    println("Finished job $(model)_$(N)_$(K) at $(Dates.now())")
end

Starting 6 jobs at 2024-02-07T17:28:22.462

Finished job scheffe_2_6_3 at 2024-02-07T17:28:44.452


Finished job scheffe_2_7_3 at 2024-02-07T17:29:05.698

Finished job scheffe_2_8_3 at 2024-02-07T17:29:30.543


Finished job special_cubic_6_3 at 2024-02-07T17:29:31.955


Finished job special_cubic_7_3 at 2024-02-07T17:29:55.269


Finished job special_cubic_8_3 at 2024-02-07T17:30:22.887


In [ ]:
model_builder = ModelBuilder.scheffe(2)
init = DesignInitializer.initializer(7, 3, model_builder, type="mixture")
scorer = OptimalityCriterion.i_criterion(model_builder, model="scheffe_2")

designs = init(10_000_000)
models = model_builder(designs)
scores = scorer(designs)

In [5]:
h5write("../../../data/random_designs.h5", "designs_scheffe_2_6_3", designs)
h5write("../../../data/random_designs.h5", "scores_scheffe_2_6_3", scores)

In [19]:
file = h5open("../../../data/random_designs.h5", "r")
for k in keys(file)
    println(k)
end

designs_scheffe_2_6_3
designs_scheffe_2_7_3
scores_scheffe_2_6_3
scores_scheffe_2_7_3


In [14]:
file = h5open("../../../data/random_designs.h5", "r")
scores = read(file, "scores_scheffe_2_7_3")
rand_designs = read(file, "designs_scheffe_2_7_3")
close(file)

In [15]:
min_score, min_idx = findmin(scores)

(0.612632917151271, 7277861)

In [16]:
rand_designs[min_idx, :, :]

7×3 Matrix{Float64}:
 0.932867   0.0508465  0.0162868
 0.0121395  0.980692   0.00716893
 0.0437565  0.51234    0.443903
 0.0605962  0.0572674  0.882136
 0.565443   0.0350885  0.399469
 0.270124   0.378659   0.351217
 0.511407   0.419125   0.0694677

In [17]:
file = h5open("../../../data/i_optimal_designs.h5", "r")
X = read(file["designs_scheffe_2_7_3"])
meta = read(file["meta_scheffe_2_7_3"])
close(file)

# X[1, : , :]
meta[1, :,:]

4×1 Matrix{Float64}:
  50.0
  50.0
 100.0
   0.5042224097471181

In [21]:
file = h5open("../../../data/i_optimal_designs.h5", "r")
for k in keys(file)
    println(k)
end
close(file)

# X[1, : , :]
# meta[1, :,:]

designs_scheffe_2_6_3
designs_scheffe_2_7_3
designs_scheffe_2_8_3
designs_scheffe_2_9_3
designs_special_cubic_10_3
designs_special_cubic_6_3
designs_special_cubic_7_3
designs_special_cubic_8_3
designs_special_cubic_9_3
meta_scheffe_2_6_3
meta_scheffe_2_7_3
meta_scheffe_2_8_3
meta_scheffe_2_9_3
meta_special_cubic_10_3
meta_special_cubic_6_3
meta_special_cubic_7_3
meta_special_cubic_8_3
meta_special_cubic_9_3
